<a href="https://colab.research.google.com/github/Mhobo/SimNets/blob/main/SimNets1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchinfo
!pip install torch
!pip install sklearn

# For visualisation: - uses this https://towardsdatascience.com/visualizing-artificial-neural-networks-anns-with-just-one-line-of-code-b4233607209e#:~:text=ANN%20Visualizer%20is%20a%20python,neural%20network%20you're%20building.
# Needs NN to have attribute 'layers', could manipulate code to do this but idk if thats what I want (model current has attributes ''layer1' 'layer2' etc.)
!pip3 install keras
!pip3 install ann_visualizer
!pip install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ann-visualizer: filename=ann_visualizer-2.5-py3-none-any.whl size=4168 sha256=42b67d3ffe65317ac77da431f37175404aca5f70103463bf7eef2e0a9e2d3f5f
  Stored in directory: /root/.cache/pip/wheels/1b/fc/58/2ab1c3b30350105929308becddda4fb59b1358e54f985e1f4a
Successfully built ann-visualizer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchinfo import summary
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from torch.functional import norm

In [64]:
device = torch.device('cpu')
num_epochs = 10
learning_rate = 0.001
reg_constant = 0.000005
L_norm = 1

In [ ]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model1 = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model2 = model1


In [ ]:
# Image preprocessing modules - CIFAR
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset1 = torchvision.datasets.CIFAR10(root='data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset1 = torchvision.datasets.CIFAR10(root='data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader1 = torch.utils.data.DataLoader(dataset=train_dataset1,
                                           batch_size=100, 
                                           shuffle=True)

test_loader1 = torch.utils.data.DataLoader(dataset=test_dataset1,
                                          batch_size=100, 
                                          shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/


In [ ]:
# Image preprocessing modules - EuroSAT
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# EuroSAT dataset - need to split into training data and test data
TEST_SIZE = 0.15 #similar to CIFAR
SEED = 42
data2 = torchvision.datasets.EuroSAT(root='data/',
                                             transform=transform,
                                             download=True)

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    range(len(data2)),
    data2.targets,
    stratify=data2.targets,
    test_size=TEST_SIZE,
    random_state=SEED)

#Putting them into train/test
train_dataset2 = Subset(data2, train_indices)
test_dataset2 = Subset(data2, test_indices)

# Data loader
train_loader2 = torch.utils.data.DataLoader(dataset=train_dataset2,
                                           batch_size=100, 
                                           shuffle=True)

test_loader2 = torch.utils.data.DataLoader(dataset=test_dataset2,
                                          batch_size=100, 
                                          shuffle=False)

  0%|          | 0/94280567 [00:00<?, ?it/s]

Extracting data/eurosat/EuroSAT.zip to data/eurosat


In [ ]:
# Loss and optimizer - Need to be careful here, something going on here
criterion = nn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(model1.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()
optimizer2 = torch.optim.Adam(model2.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [ ]:
# Train the model -Here is the meat: train them simultaneously (so inside epoch loop), optimising for simmilarity
# Want to maybe change the regularising coefficient over time? Start by prioritising being a good classifer and then make the regulariser matter more/ making it flip between big and small regulasriser
# CUrrently it ends with both models being exactly the same, but only being good at one of the tasks
# May want to decrease regularising constant, do the thing above with flipping regularising constant,may even out after a couple epochs once it learns to how to classify CIFAR, could multiply CIFAR loss by something 
# Maybe also do the thing where I only define the loss by the conv nets? Should be easy

m1w = torch.nn.utils.parameters_to_vector(model1.parameters())
m2w = torch.nn.utils.parameters_to_vector(model2.parameters())

total_step = min(len(train_loader1),len(train_loader2)) #need to maake sure this is same for both models?
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader1):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model1(images)
        loss = criterion(outputs, labels) + reg_constant*norm(m2w - torch.nn.utils.parameters_to_vector(model1.parameters()),L_norm) #add + regularisation (but only on conv layers not lin layers)

        # Backward and optimize
        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()

        if (i+1) % 100 == 0:
            print ("Cifar Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer1, curr_lr)

    #Store New Weights
    m1w = torch.nn.utils.parameters_to_vector(model1.parameters())
     
    # For EuroSAT
    for i, (images, labels) in enumerate(train_loader2):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model2(images)
        loss = criterion(outputs, labels)  + reg_constant*norm(m1w - torch.nn.utils.parameters_to_vector(model2.parameters()),L_norm) #add + regularisation (but only on conv layers not lin layers)

        # Backward and optimize
        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()

        if (i+1) % 100 == 0:
            print ("EuroSAT Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer2, curr_lr)

    #Store New Weights
    m2w = torch.nn.utils.parameters_to_vector(model2.parameters())

Cifar Epoch [1/10], Step [100/230] Loss: 2.3383
Cifar Epoch [1/10], Step [200/230] Loss: 2.2757
Cifar Epoch [1/10], Step [300/230] Loss: 2.3620
Cifar Epoch [1/10], Step [400/230] Loss: 2.4239
Cifar Epoch [1/10], Step [500/230] Loss: 2.3715
EuroSAT Epoch [1/10], Step [100/230] Loss: 2.3233
EuroSAT Epoch [1/10], Step [200/230] Loss: 2.3881
Cifar Epoch [2/10], Step [100/230] Loss: 2.3947
Cifar Epoch [2/10], Step [200/230] Loss: 2.4406
Cifar Epoch [2/10], Step [300/230] Loss: 2.3983
Cifar Epoch [2/10], Step [400/230] Loss: 2.3446
Cifar Epoch [2/10], Step [500/230] Loss: 2.4115
EuroSAT Epoch [2/10], Step [100/230] Loss: 2.3277
EuroSAT Epoch [2/10], Step [200/230] Loss: 2.3668
Cifar Epoch [3/10], Step [100/230] Loss: 2.4540
Cifar Epoch [3/10], Step [200/230] Loss: 2.4327
Cifar Epoch [3/10], Step [300/230] Loss: 2.3895
Cifar Epoch [3/10], Step [400/230] Loss: 2.4129
Cifar Epoch [3/10], Step [500/230] Loss: 2.3717
EuroSAT Epoch [3/10], Step [100/230] Loss: 2.3444
EuroSAT Epoch [3/10], Step [20

In [ ]:
# Test the model
model1.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader1:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the CIFAR model on the test images: {} %'.format(100 * correct / total))

    model2.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader2:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the EuroSAT model on the test images: {} %'.format(100 * correct / total))

Accuracy of the CIFAR model on the test images: 10.57 %
Accuracy of the EuroSAT model on the test images: 9.876543209876543 %


In [ ]:
#Save the model checkpoint
torch.save(model1.state_dict(), 'resnet.ckpt')
torch.save(model2.state_dict(), 'resnet.ckpt')

In [ ]:
summary(model1,input_size = (1,3,32,32), verbose= 2)
model1.parameters

In [ ]:
norm(m1w - m2w)
max_dist = 4
[i for i,v in enumerate(abs(m1w - m2w)) if v > max_dist] #find indices of edges where differnces in weights are more than a thing
# Would be great if I now know how to taken those indices and figure out where they are in terms of layer and stuff

[]

In [ ]:
# model2.layers # Error but ResNet should have attribute layers from its definiton? Thats in init only, think that function does something fancy
# Make a copy of it that does have attribute layers which is an array of all the layers
class ResNetL(ResNet):
  pass 

In [ ]:
dir(ResNet)

In [ ]:
from ann_visualizer.visualize import ann_viz;

ann_viz(model1, title="My first neural network")

AttributeError: ignored